In [ ]:
#!pip install sharepy
import sharepy 
import os
#System variables
MAIL_NUB = os.environ.get('MAIL_NUB')
PASS_MAIL_NUB = os.environ.get('PASS_MAIL_NUB')
s = sharepy.connect("nubsieteocho.sharepoint.com", MAIL_NUB, PASS_MAIL_NUB)

In [ ]:
import smtplib
import pandas as pd
from datetime import datetime, timedelta
from pandas import Timestamp

from email import encoders
from os.path import basename
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from string import printable

In [ ]:
#import data
p = s.getfile("https://nubsieteocho.sharepoint.com/sites/ODAOPS/Shared%20Documents/General/CAB%20Trackers/Daily%20CAB%20Tracker.xlsx",
              filename='Daily CAB Tracker.xlsx')
df_compass = pd.read_excel('Daily CAB Tracker.xlsx', sheet_name='Tracker')
df_compass.dropna(axis=0, how='all', subset=['Request ID', 'Customer Name'], inplace=True)


In [ ]:
# SMTP Mail variables
today = datetime.strftime(datetime.today(), '%m-%d-%Y')
myMail = 'testNub78@outlook.com'
password = 'EasyP4ssword78'
subjectEmail = 'Coverage error report'

msg1 = """<h3>Hola, %s</h3>
        <p><span style="color: rgb(71, 85, 119);">Parece que tienes varios errores en la informaci&oacute;n registrada en tus cuentas.</span></p>
        <p><span style="color: rgb(71, 85, 119);">Asegurate de revisar el archivo adjunto para validar los campos que est&aacute;n generando conflictos.</span></p>
        <p><span style="color: rgb(71, 85, 119);">Feliz D&iacute;a.</span></p>"""


In [ ]:
def format_date_column(serie):
    format_date = []
    for index, value in serie.iteritems():
        if type(value) != datetime and type(value) != Timestamp:
            if pd.isnull(value) or value != value:
                # Fechas vacias
                value = datetime.strptime(
                    '2030-2-2 00:00:00', '%Y-%m-%d %H:%M:%S')
                format_date.append(value)
            else:
                # Error de formato
                value = datetime.strptime(
                    '2050-2-2 00:00:00', '%Y-%m-%d %H:%M:%S')
                format_date.append(value)
        else:
            # Fechas normales
            value = datetime.strptime(str(value), '%Y-%m-%d %H:%M:%S')
            format_date.append(value)

    return format_date


In [ ]:
# Advisor // Advisor Name // Date Advisor Assigned
Advisor = df_compass[['Request ID', 'RR (internal)', 'Advisor', 'Advisor Name', 'Date Advisor Assigned']].copy()
Advisor = Advisor[Advisor['RR (internal)'] == 'Yes']
Advisor['Advisor'] = Advisor['Advisor'].apply(lambda x: None if pd.isna(x) == False else 'Empty value')
Advisor['Advisor Name'] = Advisor['Advisor Name'].apply(lambda x: None if pd.isna(x) == False else 'Empty value')
Advisor['Date Advisor Assigned'] = Advisor['Date Advisor Assigned'].apply(lambda x: None if pd.isna(x) == False else 'Empty value')
Advisor.dropna(axis=0, how='all', subset=['Advisor', 'Advisor Name', 'Date Advisor Assigned'], inplace=True)
Advisor.drop(axis=1, columns=['RR (internal)'], inplace=True)

In [ ]:
# Date Readaout Done // Cisco.

dat_rdone = df_compass[['Request ID', 'Date Advisor Assigned', 'Date Readout Done']].copy()
dat_rdone['Date Advisor Assigned'] = format_date_column(dat_rdone['Date Advisor Assigned']) 
dat_rdone['Date Readout Done'] = format_date_column(dat_rdone['Date Readout Done']) 

for id, row in dat_rdone.iterrows():
    if row['Date Readout Done'] != Timestamp('2030-2-2 00:00:00') and row['Date Readout Done'] != Timestamp('2050-2-2 00:00:00'):
        if row['Date Advisor Assigned'] != Timestamp('2030-2-2 00:00:00') and row['Date Advisor Assigned'] != Timestamp('2050-2-2 00:00:00'):
            if row['Date Advisor Assigned'] > row['Date Readout Done']:
                dat_rdone.loc[id, 'Date Readout Done'] = 'Date Advisor Assigned greater'
            else:
                dat_rdone.loc[id, 'Date Readout Done'] = None
        else:
            dat_rdone.loc[id, 'Date Readout Done'] = None 
    elif row['Date Readout Done'] == Timestamp('2030-2-2 00:00:00') and row['Date Advisor Assigned'] != Timestamp('2030-2-2 00:00:00'):
        dat_rdone.loc[id, 'Date Readout Done'] = 'Empty Date'
    elif row['Date Readout Done'] == Timestamp('2050-2-2 00:00:00'):
        dat_rdone.loc[id, 'Date Readout Done'] = 'Format error'
    else:
        dat_rdone.loc[id, 'Date Readout Done'] = None

dat_rdone.dropna(axis=0, subset=['Date Readout Done'], inplace=True)
dat_rdone.drop(axis=1, columns=['Date Advisor Assigned'], inplace=True)
dat_rdone

In [ ]:
# Readout delivery status

del_status = df_compass[['Request ID', 'Date Readout Done', 'Readout Delivery Status']].copy()
del_status = del_status[(pd.notna(del_status['Date Readout Done'])) & (pd.isna(del_status['Readout Delivery Status']))]
del_status['Readout Delivery Status'] = 'Empty value'
del_status.drop(axis=1, columns=['Date Readout Done'], inplace=True)

In [ ]:
# Reason Declined // Item recommended

declined = df_compass[['Request ID', 'Readout Delivery Status', 'Reason Declined']]
declined = declined[declined['Readout Delivery Status'] == 'Declined']
declined = declined[pd.isna(declined['Reason Declined'])]
declined['Reason Declined'] = 'Empty value'
declined.drop(axis=1, columns=['Readout Delivery Status'], inplace=True)

item = df_compass[['Request ID', 'Readout Delivery Status', 'Items Recommended']].copy()
item = item[(item['Readout Delivery Status'] == 'Complete') | (item['Readout Delivery Status'] == 'Emailed Recommendations')]
item = item[pd.isna(item['Items Recommended'])]
item['Items Recommended'] = 'Empty value'
item.drop(axis=1, columns=['Readout Delivery Status'], inplace=True)

res_declined = declined.merge(item, how='outer', left_on='Request ID', right_on='Request ID')

In [ ]:
# Mayor Renewal QTR

ren_qtr = df_compass[['Request ID', 'Major Renewal QTR', 'Date Presentation Created']] 
ren_qtr = ren_qtr[pd.isna(ren_qtr['Major Renewal QTR']) & pd.notna(ren_qtr['Date Presentation Created'])]
ren_qtr.fillna('Empty value', inplace=True)
ren_qtr.drop(axis=1, columns='Date Presentation Created', inplace=True)

In [ ]:
# Date presentation Created
# Note: The column date readout done do not have the correct format (it's string)

pre_created = df_compass[['Request ID', 'Date Advisor Assigned', 'Date Presentation Created']].copy()
pre_created['Date Advisor Assigned'] = format_date_column(pre_created['Date Advisor Assigned']) 
pre_created['Date Presentation Created'] = format_date_column(pre_created['Date Presentation Created']) 
pre_created = pre_created[(pre_created['Date Presentation Created'] < pre_created['Date Advisor Assigned'])]
pre_created.drop(labels=pre_created[pre_created['Date Advisor Assigned'] == '2030-02-02'].index, axis=0, inplace=True)
pre_created.drop(labels=pre_created[pre_created['Date Advisor Assigned'] == '2050-02-02'].index, axis=0, inplace=True)
pre_created['Date Presentation Created'] = 'Date Advisor Assigned is greater'
pre_created.drop(axis=1, columns=['Date Advisor Assigned'], inplace=True)

In [ ]:
# Presentation Creation Time (min)

pre_created_tim = df_compass[['Request ID', 'Date Presentation Created', 'Presentation Creation Time (min)']].copy()
pre_created_tim = pre_created_tim[(pd.notna(pre_created_tim['Date Presentation Created'])) & (pd.isna(pre_created_tim['Presentation Creation Time (min)']))]
pre_created_tim['Presentation Creation Time (min)'] = 'Empty value'
pre_created_tim.drop(axis=1, columns=['Date Presentation Created'], inplace=True)

In [ ]:
# Readout Duration (min)

re_duration_m = df_compass[['Request ID', 'Readout Delivery Status', 'Readout Duration (min)']].copy()
re_duration_m = re_duration_m[(re_duration_m['Readout Delivery Status'] == 'Complete') | (re_duration_m['Readout Delivery Status'] == 'Emailed Recommendations')]
re_duration_m = re_duration_m[pd.isna(re_duration_m['Readout Duration (min)'])]
re_duration_m['Readout Duration (min)'] = 'Empty value'
re_duration_m.drop(axis=1, columns=['Readout Delivery Status'], inplace=True)

In [ ]:
# Date Invite Sent

dat_invite_s = df_compass[['Request ID', 'Date Advisor Assigned', 'Date Invite Sent']].copy()
dat_invite_s['Date Advisor Assigned'] = format_date_column(dat_invite_s['Date Advisor Assigned']) 
dat_invite_s['Date Invite Sent'] = format_date_column(dat_invite_s['Date Invite Sent'])
dat_invite_s = dat_invite_s[dat_invite_s['Date Advisor Assigned'] > dat_invite_s['Date Invite Sent']]
dat_invite_s.drop(labels=dat_invite_s[dat_invite_s['Date Advisor Assigned'] == '2030-02-02'].index, axis=0, inplace=True)
dat_invite_s.drop(labels=dat_invite_s[dat_invite_s['Date Advisor Assigned'] == '2050-02-02'].index, axis=0, inplace=True)
dat_invite_s['Date Invite Sent'] = 'Date Advisor Assigned greater'
dat_invite_s.drop(axis=1, columns=['Date Advisor Assigned'], inplace=True)

In [ ]:
# Date Summary Sent
# En pausa dado que la columna readout done tiene formato string

In [ ]:
# IB value (in M) // Cover the Uncovered Presented

ibvalue_and_cover = df_compass[['Request ID', 'Date Presentation Created', 'IB value (in M)', 'Cover the Uncovered Presented']].copy()
ibvalue_and_cover = ibvalue_and_cover[pd.notna(ibvalue_and_cover['Date Presentation Created'])]
ibvalue_and_cover = ibvalue_and_cover[(pd.isna(ibvalue_and_cover['IB value (in M)'])) | (pd.isna(ibvalue_and_cover['Cover the Uncovered Presented']))]
ibvalue_and_cover.fillna('Empty value', inplace=True)
ibvalue_and_cover['Cover the Uncovered Presented'] = ibvalue_and_cover['Cover the Uncovered Presented'].apply(
    lambda x: None if x != 'Empty value' else x)
ibvalue_and_cover['IB value (in M)'] = ibvalue_and_cover['IB value (in M)'].apply(
    lambda x: None if x != 'Empty value' else x)
ibvalue_and_cover.drop(axis=1, columns=['Date Presentation Created'], inplace=True)

In [ ]:
# AT Interested (CTU)

at_interested = df_compass[['Request ID', 'Readout Delivery Status', 'AT Interested (CTU)']].copy()
at_interested = at_interested[(at_interested['Readout Delivery Status'] == 'Complete') &
                pd.isnull(at_interested['AT Interested (CTU)'])]
at_interested['AT Interested (CTU)'] = 'Empty value'
at_interested.drop(axis=1, columns=['Readout Delivery Status'], inplace=True)

In [ ]:
# Uplift to SSPT In Pursue

up_sspt = df_compass[['Request ID', 'Date Presentation Created', 'Uplift to SSPT In Pursue']].copy()
up_sspt = up_sspt[pd.notna(up_sspt['Date Presentation Created'])]
up_sspt = up_sspt[pd.isna(up_sspt['Uplift to SSPT In Pursue'])]
up_sspt['Uplift to SSPT In Pursue'] = 'Empty value'
up_sspt.drop(axis=1, columns=['Date Presentation Created'], inplace=True)

In [ ]:
# AT Interested (SSPT)

at_interested_sspt = df_compass[['Request ID', 'Readout Delivery Status', 'AT Interested (SSPT)']].copy()
at_interested_sspt = at_interested_sspt[(at_interested_sspt['Readout Delivery Status'] == 'Complete') &
                pd.isnull(at_interested_sspt['AT Interested (SSPT)'])]
at_interested_sspt['AT Interested (SSPT)'] = 'Empty value'
at_interested_sspt.drop(axis=1, columns=['Readout Delivery Status'], inplace=True)

In [ ]:
# DNA with Success Tracks Level 2 Presented

dna_presented = df_compass[['Request ID', 'Date Presentation Created', 'DNA with Success Tracks Level 2 Presented']].copy()
dna_presented = dna_presented[pd.notna(dna_presented['Date Presentation Created'])]
dna_presented = dna_presented[pd.isna(dna_presented['DNA with Success Tracks Level 2 Presented'])]
dna_presented['DNA with Success Tracks Level 2 Presented'] = 'Empty value'
dna_presented.drop(axis=1, columns=['Date Presentation Created'], inplace=True)

In [ ]:
# AT Interested (ST)

at_interested_st = df_compass[['Request ID', 'Readout Delivery Status', 'AT Interested (ST)']].copy()
at_interested_st = at_interested_st[(at_interested_st['Readout Delivery Status'] == 'Complete') &
                pd.isnull(at_interested_st['AT Interested (ST)'])]
at_interested_st['AT Interested (ST)'] = 'Empty value'
at_interested_st.drop(axis=1, columns=['Readout Delivery Status'], inplace=True)

In [ ]:
# SW Support Presented

sw_presented = df_compass[['Request ID', 'Date Presentation Created', 'SW Support Presented']].copy()
sw_presented = sw_presented[pd.notna(sw_presented['Date Presentation Created'])]
sw_presented = sw_presented[pd.isna(sw_presented['SW Support Presented'])]
sw_presented['SW Support Presented'] = 'Empty value'
sw_presented.drop(axis=1, columns=['Date Presentation Created'], inplace=True)

In [ ]:
# AT Interested (SW)

at_interested_sw = df_compass[['Request ID', 'Readout Delivery Status', 'AT Interested (SW)']].copy()
at_interested_sw = at_interested_sw[(at_interested_sw['Readout Delivery Status'] == 'Complete') &
                pd.isnull(at_interested_sw['AT Interested (SW)'])]
at_interested_sw['AT Interested (SW)'] = 'Empty value'
at_interested_sw.drop(axis=1, columns=['Readout Delivery Status'], inplace=True)

In [ ]:
# Expert Care Presented

exp_presented = df_compass[['Request ID', 'Date Presentation Created', 'Expert Care Presented']].copy()
exp_presented = exp_presented[pd.notna(exp_presented['Date Presentation Created'])]
exp_presented = exp_presented[pd.isna(exp_presented['Expert Care Presented'])]
exp_presented['Expert Care Presented'] = 'Empty value'
exp_presented.drop(axis=1, columns=['Date Presentation Created'], inplace=True)

In [ ]:
# AT Interested (EC)

at_interested_ec = df_compass[['Request ID', 'Readout Delivery Status', 'AT Interested (EC)']].copy()
at_interested_ec = at_interested_ec[(at_interested_ec['Readout Delivery Status'] == 'Complete') &
                pd.isnull(at_interested_ec['AT Interested (EC)'])]
at_interested_ec['AT Interested (EC)'] = 'Empty value'
at_interested_ec.drop(axis=1, columns=['Readout Delivery Status'], inplace=True)

In [ ]:
# BCS 3.0 Presented

bcs3_presented = df_compass[['Request ID', 'Date Presentation Created', 'BCS 3.0 Presented']].copy()
bcs3_presented = bcs3_presented[pd.notna(bcs3_presented['Date Presentation Created'])]
bcs3_presented = bcs3_presented[pd.isna(bcs3_presented['BCS 3.0 Presented'])]
bcs3_presented['BCS 3.0 Presented'] = 'Empty value'
bcs3_presented.drop(axis=1, columns=['Date Presentation Created'], inplace=True)

In [ ]:
# BCS - IB based

bcs_ib_bas = df_compass[['Request ID', 'Date Presentation Created', 'BCS - IB based']].copy()
bcs_ib_bas = bcs_ib_bas[pd.notna(bcs_ib_bas['Date Presentation Created'])]
bcs_ib_bas = bcs_ib_bas[pd.isna(bcs_ib_bas['BCS - IB based'])]
bcs_ib_bas['BCS - IB based'] = 'Empty value'
bcs_ib_bas.drop(axis=1, columns=['Date Presentation Created'], inplace=True)

In [ ]:
# AT Interested (BCS)

at_interested_bcs = df_compass[['Request ID', 'Readout Delivery Status', 'AT Interested (BCS)']].copy()
at_interested_bcs = at_interested_bcs[(at_interested_bcs['Readout Delivery Status'] == 'Complete') &
                pd.isnull(at_interested_bcs['AT Interested (BCS)'])]
at_interested_bcs['AT Interested (BCS)'] = 'Empty value'
at_interested_bcs.drop(axis=1, columns=['Readout Delivery Status'], inplace=True)

In [ ]:
# IRR Presented

irr_presented = df_compass[['Request ID', 'Date Presentation Created', 'IRR Presented']].copy()
irr_presented = irr_presented[pd.notna(irr_presented['Date Presentation Created'])]
irr_presented = irr_presented[pd.isna(irr_presented['IRR Presented'])]
irr_presented['IRR Presented'] = 'Empty value'
irr_presented.drop(axis=1, columns=['Date Presentation Created'], inplace=True)

In [ ]:
# AT Interested (IRR)

at_interested_irr = df_compass[['Request ID', 'Readout Delivery Status', 'AT Interested (IRR)']].copy()
at_interested_irr = at_interested_irr[(at_interested_irr['Readout Delivery Status'] == 'Complete') &
                pd.isnull(at_interested_irr['AT Interested (IRR)'])]
at_interested_irr['AT Interested (IRR)'] = 'Empty value'
at_interested_irr.drop(axis=1, columns=['Readout Delivery Status'], inplace=True)

In [ ]:
# PR Presented

pr_presented = df_compass[['Request ID', 'Date Presentation Created', 'PR Presented']].copy()
pr_presented = pr_presented[pd.notna(pr_presented['Date Presentation Created'])]
pr_presented = pr_presented[pd.isna(pr_presented['PR Presented'])]
pr_presented['PR Presented'] = 'Empty value'
pr_presented.drop(axis=1, columns=['Date Presentation Created'], inplace=True)

In [ ]:
# AT Interested (PR)

at_interested_pr = df_compass[['Request ID', 'Readout Delivery Status', 'AT Interested (PR)']].copy()
at_interested_pr = at_interested_pr[(at_interested_pr['Readout Delivery Status'] == 'Complete') &
                pd.isnull(at_interested_pr['AT Interested (PR)'])]
at_interested_pr['AT Interested (PR)'] = 'Empty value'
at_interested_pr.drop(axis=1, columns=['Readout Delivery Status'], inplace=True)

In [ ]:
# Closing Date
# En pausa dado que la columna readout done tiene formato string

clos_date = df_compass[['Request ID', 'Date Readout Done', 'Closing Date']].copy()

In [ ]:
# Summary Slide Link

sum_link = df_compass[['Request ID', 'Date Presentation Created', 'Summary Slide Link']].copy()
sum_link = sum_link[pd.notna(sum_link['Date Presentation Created'])]
sum_link = sum_link[pd.isna(sum_link['Summary Slide Link'])]
sum_link['Summary Slide Link'] = 'Empty value'
sum_link.drop(axis=1, columns=['Date Presentation Created'], inplace=True)

In [ ]:
# Account Status // Account Stage

account_st = df_compass[['Request ID', 'Readout Delivery Status', 'Account Status', 'Account Stage']].copy()


In [ ]:
# Declined Reason Lvl2

dec_reason_2 = df_compass[['Request ID', 'Readout Delivery Status', 'Declined Reason Lvl2']].copy()
dec_reason_2 = dec_reason_2[dec_reason_2['Readout Delivery Status'] == 'Declined']
dec_reason_2 = dec_reason_2[pd.isna(dec_reason_2['Declined Reason Lvl2'])]
dec_reason_2['Declined Reason Lvl2'] = 'Empty value'
dec_reason_2.drop(axis=1, columns=['Readout Delivery Status'], inplace=True)

In [ ]:
# Reason On Hold

res_hold = df_compass[['Request ID', '# Days on Hold (Analysis)', '# Days on Hold (Workshop)', 'Reason On Hold']].copy()
res_hold = res_hold[(pd.notna(res_hold['# Days on Hold (Analysis)'])) | (pd.notna(res_hold['# Days on Hold (Workshop)']))]
res_hold = res_hold[pd.isna(res_hold['Reason On Hold'])]
res_hold['Reason On Hold'] = 'Empty value'
res_hold.drop(axis=1, columns=['# Days on Hold (Analysis)'], inplace=True)
res_hold.drop(axis=1, columns=['# Days on Hold (Workshop)'], inplace=True)

In [ ]:
# Merge

final_report = Advisor.merge(dat_rdone, how='outer', left_on='Request ID', right_on='Request ID').merge(
    del_status, how='outer', left_on='Request ID', right_on='Request ID').merge(
    res_declined, how='outer', left_on='Request ID', right_on='Request ID').merge(
    ren_qtr, how='outer', left_on='Request ID', right_on='Request ID').merge(
    pre_created, how='outer', left_on='Request ID', right_on='Request ID').merge(
    pre_created_tim, how='outer', left_on='Request ID', right_on='Request ID').merge(
    re_duration_m, how='outer', left_on='Request ID', right_on='Request ID').merge(
    dat_invite_s, how='outer', left_on='Request ID', right_on='Request ID').merge(
    ibvalue_and_cover, how='outer', left_on='Request ID', right_on='Request ID').merge(
    at_interested, how='outer', left_on='Request ID', right_on='Request ID').merge(
    up_sspt, how='outer', left_on='Request ID', right_on='Request ID').merge(
    at_interested_sspt, how='outer', left_on='Request ID', right_on='Request ID').merge(
    dna_presented, how='outer', left_on='Request ID', right_on='Request ID').merge(
    at_interested_st, how='outer', left_on='Request ID', right_on='Request ID').merge(
    sw_presented, how='outer', left_on='Request ID', right_on='Request ID').merge(
    at_interested_sw, how='outer', left_on='Request ID', right_on='Request ID').merge(
    exp_presented, how='outer', left_on='Request ID', right_on='Request ID').merge(
    at_interested_ec, how='outer', left_on='Request ID', right_on='Request ID').merge(
    bcs3_presented, how='outer', left_on='Request ID', right_on='Request ID').merge( 
    bcs_ib_bas, how='outer', left_on='Request ID', right_on='Request ID').merge(
    at_interested_bcs, how='outer', left_on='Request ID', right_on='Request ID').merge(
    irr_presented, how='outer', left_on='Request ID', right_on='Request ID').merge(
    at_interested_irr, how='outer', left_on='Request ID', right_on='Request ID').merge(
    pr_presented, how='outer', left_on='Request ID', right_on='Request ID').merge(
    at_interested_pr, how='outer', left_on='Request ID', right_on='Request ID').merge(
    sum_link, how='outer', left_on='Request ID', right_on='Request ID').merge(
    dec_reason_2, how='outer', left_on='Request ID', right_on='Request ID').merge(
    res_hold, how='outer', left_on='Request ID', right_on='Request ID')

In [ ]:
final_report